# mini_proj_03_20171541_차영민

In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

## kc_house_data 받아오기

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("kc_house_data.csv")

## kc_house_data의 type과 특성값들이 전부 존재하는지 확인

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

1. id = ID
2. date = 거래날짜
3. price = 거래가격(예측해야 되는 것)
4. bedrooms = 침실수
5. bathrooms = 욕실수
6. sqft_living = 내부면적
7. sqft_lot = 땅면적
8. floors = 층수
9. waterfront = 해변조망
10. view = 경관
11. condition = 상태
12. grade = 구조설계
13. sqft_above = 위층면적
14. sqft_basement = 지하면적
15. yr_built = 건축연도
16. yr_renovated = 증축연도
17. zipcode = 우편주소
18. lat = 위도
19. long = 경도
20. sqft_living15 = 주위 15가구 내부 면적?
21. sqft_lot15 = 주위 15가구 외부 면적?

## date는 object이므로 숫자로 변환 필요, 모든 특성 개수 21613개 이므로 결측치 존재X

# 특성조합

In [4]:
df["date"] = pd.to_datetime(df["date"])
import datetime #현재 날짜 받아오기 위해 datatime 사용
df["now"] = pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d"))
df["now-date"] = (df["now"] - df["date"]).dt.days
df["now-date"] #현재 날짜 - 거래날짜를 int형으로 변환

0        2603
1        2546
2        2468
3        2546
4        2475
         ... 
21608    2748
21609    2470
21610    2715
21611    2508
21612    2601
Name: now-date, Length: 21613, dtype: int64

In [5]:
df["gps"] = df["long"] / df["lat"]
df["backyard"] = df["sqft_lot"] - df["sqft_living"]
df["sqft_rate"] = df["sqft_lot"] - df["sqft_above"]
df["beach"] = df["waterfront"] + df["lat"]
df["view + long"] = df["view"] + df["long"]
df["grade**2"] = df["grade"]**2
df["bedrooms / sqft_living"] = df["bedrooms"] / df["sqft_living"]
df["zipcode / long"] = df["zipcode"] / df["long"]
df["living"] = df["sqft_living"] + df["sqft_living15"]
df["grade**2 / now-date"] = df["grade**2"] / df["now-date"]
df["waterfront * bathrooms"] = df["waterfront"] * df["bathrooms"]
df["condition**2"] = df["condition"]**2

## price와 표쥰 상관 계수 조사

In [6]:
corr_matrix = df.corr()
corr_matrix["price"].sort_values(ascending=False)

price                     1.000000
sqft_living               0.702035
living                    0.694920
grade**2                  0.692402
grade**2 / now-date       0.688335
grade                     0.667434
sqft_above                0.605567
sqft_living15             0.585379
bathrooms                 0.525138
beach                     0.404086
view + long               0.400277
view                      0.397293
sqft_basement             0.323816
waterfront * bathrooms    0.318485
gps                       0.308589
bedrooms                  0.308350
lat                       0.307003
waterfront                0.266369
floors                    0.256794
yr_renovated              0.126434
sqft_lot                  0.089661
sqft_lot15                0.082447
sqft_rate                 0.077825
backyard                  0.074361
yr_built                  0.054012
condition**2              0.037287
condition                 0.036362
long                      0.021626
zipcode / long      

## x_train과 y_train(housing_label) 선언

### 선언 후, x_train이 잘 나왔는지 확인

In [7]:
x_train = df.drop(["price", "date", "condition", "id", "floors", "bedrooms", "sqft_basement", "now", "view", "yr_renovated", "zipcode"], axis = 1)
housing_label = df["price"].copy()
x_train

,bathrooms,sqft_living,sqft_lot,waterfront,grade,sqft_above,yr_built,lat,long,sqft_living15,...,sqft_rate,beach,view + long,grade**2,bedrooms / sqft_living,zipcode / long,living,grade**2 / now-date,waterfront * bathrooms,condition**2
0,1.00,1180,5650,0,7,1180,1955,47.5112,-122.257,1340,...,4470,47.5112,-122.257,49,0.002542,-803.046042,2520,0.018824,0.0,9
1,2.25,2570,7242,0,7,2170,1951,47.7210,-122.319,1690,...,5072,47.7210,-122.319,49,0.001167,-802.205708,4260,0.019246,0.0,9
2,1.00,770,10000,0,6,770,1933,47.7379,-122.233,2720,...,9230,47.7379,-122.233,36,0.002597,-801.976553,3490,0.014587,0.0,9
3,3.00,1960,5000,0,7,1050,1965,47.5208,-122.393,1360,...,3950,47.5208,-122.393,49,0.002041,-801.810561,3320,0.019246,0.0,25
4,2.00,1680,8080,0,8,1680,1987,47.6168,-122.045,1800,...,6400,47.6168,-122.045,64,0.001786,-803.588840,3480,0.025859,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,2.50,1530,1131,0,8,1530,2009,47.6993,-122.346,1530,...,-399,47.6993,-122.346,64,0.001961,-801.848855,3060,0.023290,0.0,9
21609,2.50,2310,5813,0,8,2310,2014,47.5107,-122.362,1830,...,3503,47.5107,-122.362,64,0.001732,-802.095422,4140,0.025911,0.0,9
21610,0.75,1020,1350,0,7,1020,2009,47.5944,-122.299,1020,...,330,47.5944,-122.299,49,0.001961,-802.492253,2040,0.018048,0.0,9
21611,2.50,1600,2388,0,8,1600,2004,47.5345,-122.069,1410,...,788,47.5345,-122.069,64,0.001875,-803.045818,3010,0.025518,0.0,9


### housing_label이 잘 나왔는지 확인

In [8]:
housing_label

0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
21608    360000.0
21609    400000.0
21610    402101.0
21611    400000.0
21612    325000.0
Name: price, Length: 21613, dtype: float64

# 사용하는 모델이 Extra Trees Regressor와 GBRT이므로 정규화는 필요 X

## Grid Search(Extra Trees Regressor)

In [9]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
Ex_param_grid = {
    'n_estimators' : [700,750,800],
    'max_depth' : [29,30,31,32],
    'min_samples_leaf' : [1,2],
    'min_samples_split' : [2,3,4]
}

찾고자 하는 값이 너무 많으면, 시간이 오래 걸리기 때문에 여러번 진행 후 범위를 줄임.

In [10]:
Extra_reg = ExtraTreesRegressor(random_state=42, n_jobs = -1)
Extra_reg.fit(x_train, housing_label)

Ex_grid = GridSearchCV(Extra_reg, param_grid = Ex_param_grid, n_jobs=-1, verbose = 1)
Ex_grid.fit(x_train, housing_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=ExtraTreesRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [29, 30, 31, 32],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [700, 750, 800]},
             verbose=1)

## Grid Search 에서 나온 최상의 하이퍼파라미터를 모델에 대입 후 fit

In [11]:
print(Ex_grid.best_params_)

{'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 800}


In [12]:
Ex_max_depth = Ex_grid.best_params_["max_depth"]
Ex_min_samples_leaf = Ex_grid.best_params_["min_samples_leaf"]
Ex_min_samples_split = Ex_grid.best_params_["min_samples_split"]
Ex_n_estimators = Ex_grid.best_params_["n_estimators"]

In [13]:
Ex_reg = ExtraTreesRegressor(n_estimators = Ex_n_estimators, max_depth = Ex_max_depth, 
                                     min_samples_leaf=Ex_min_samples_leaf, min_samples_split = Ex_min_samples_split, 
                                     random_state=42, n_jobs = -1)
Ex_reg.fit(x_train, housing_label)

ExtraTreesRegressor(max_depth=30, n_estimators=800, n_jobs=-1, random_state=42)

## Gradient Boosting Regressor의 n_estimators 찾기

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt_reg = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)
gbrt_reg.fit(x_train, housing_label)

GradientBoostingRegressor(max_depth=2, random_state=42, warm_start=True)

## 10-fold cross validation의 평균 오차가 향상되지 않으면(5번) 훈련 조기 종료

train set과 test set을 나누지 않았기 때문에, rmse로는 확인 불가!(rmse가 0으로 과대적합 할 때까지 계속 된다.)
그러므로 10-fold cross validation을 하고 난 후, cross validation을 서로 비교하여 최적의 값을 나오게 하고,
모델에 대입 후 fit

In [15]:
from sklearn.model_selection import cross_val_score

def cv10(gbrt, x_train, y_train):
    gbrt_cv2_scores = cross_val_score(gbrt,x_train,y_train, scoring="neg_mean_squared_error", cv=10)
    gbrt_cv2_rmse_scores = np.sqrt(-gbrt_cv2_scores)
    mean = gbrt_cv2_rmse_scores.mean()
    return mean

#### max_depth는 2로 고정

In [16]:
min_val_error = float("inf")
error_going_up = 0
best_n_estimators = 0
for n_estimators in range(3500, 5500, 100):
    gbrt_reg.n_estimators = n_estimators
    gbrt_reg.fit(x_train, housing_label)
    mean = cv10(gbrt_reg,x_train, housing_label)
    if mean < min_val_error:
        min_val_error = mean
        error_going_up = 0
        print(f"mean: {min_val_error}, n_estimators: {n_estimators}")
        best_n_estimators = n_estimators
    else:
        print(f"over the best_n_estimators! mean: {mean} n_estimators: {n_estimators}, best_n_estimators: {best_n_estimators}")
        error_going_up += 1
        if error_going_up == 5:
            gbrt_reg.n_estimators = best_n_estimators
            break

mean: 117680.68413725015, n_estimators: 3500
mean: 117654.11132468151, n_estimators: 3600
mean: 117611.93569349959, n_estimators: 3700
mean: 117589.11818586613, n_estimators: 3800
mean: 117536.26315713013, n_estimators: 3900
mean: 117511.07606747425, n_estimators: 4000
mean: 117510.55002574774, n_estimators: 4100
over the best_n_estimators! mean: 117518.17980918265 n_estimators: 4200, best_n_estimators: 4100
over the best_n_estimators! mean: 117556.7672170349 n_estimators: 4300, best_n_estimators: 4100
over the best_n_estimators! mean: 117542.87114022669 n_estimators: 4400, best_n_estimators: 4100
over the best_n_estimators! mean: 117542.9749642502 n_estimators: 4500, best_n_estimators: 4100
over the best_n_estimators! mean: 117588.52959720837 n_estimators: 4600, best_n_estimators: 4100


## GBRT 모델 fit

In [17]:
gbrt_reg = GradientBoostingRegressor(max_depth=2, n_estimators = best_n_estimators , random_state=42)
gbrt_reg.fit(x_train, housing_label)

GradientBoostingRegressor(max_depth=2, n_estimators=4100, random_state=42)

## 각각 모델의 rmse 출력

In [18]:
def display_rmse_scores(ex_sc, gbrt_sc):
    print(f"Extra_rmse_scores : {ex_sc}")
    print(f"gert_rmse_scores : {gbrt_sc}")

In [19]:
from sklearn.metrics import mean_squared_error

Ex_predictions = Ex_reg.predict(x_train)
Ex_mse = mean_squared_error(housing_label, Ex_predictions)
Ex_rmse = np.sqrt(Ex_mse)

gbrt_predictions = gbrt_reg.predict(x_train)
gbrt_mse = mean_squared_error(housing_label, gbrt_predictions)
gbrt_rmse = np.sqrt(gbrt_mse)

display_rmse_scores(Ex_rmse, gbrt_rmse)

Extra_rmse_scores : 369.4055363670069
gert_rmse_scores : 74727.23258651365


## 10-fold cross-validation 이후 각각의 모델의 Scores, 평균오차, 표쥰편차 출력

In [20]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [21]:
Ex_scores = cross_val_score(Ex_reg, x_train, housing_label,
                                scoring="neg_mean_squared_error", cv=10)
Ex_rmse_scores = np.sqrt(-Ex_scores)

In [22]:
gbrt_scores = cross_val_score(gbrt_reg, x_train, housing_label,
                         scoring="neg_mean_squared_error", cv=10)
gbrt_rmse_scores = np.sqrt(-gbrt_scores)

In [23]:
Ten_fold_score = [Ex_rmse_scores, gbrt_rmse_scores]
for x in range(2):
    if x ==1:
        print("Extra Trees Regression's score!")
    elif x ==2:
        print("Gradient boosting's score!")

    display_scores(Ten_fold_score[x])
    print("-------------------------------------------------------------------------")

Scores: [122085.41382816 135756.70855113 110131.92155508 114624.83043031
 101354.08868651 123909.34112597 125893.69870206 116461.48333848
 116051.06439364 126008.13511887]
Mean: 119227.6685730201
Standard deviation: 9136.90756383756
-------------------------------------------------------------------------
Extra Trees Regression's score!
Scores: [118375.06790405 124244.50235326 109592.16504442 120320.13238175
 104032.47681141 123696.72231018 121376.38272663 108081.84712313
 112937.20578383 132448.99781881]
Mean: 117510.55002574774
Standard deviation: 8273.651339407332
-------------------------------------------------------------------------


# voting(엑스트라 트리, 그레디언트 부스팅)

In [24]:
from sklearn.ensemble import VotingRegressor

voting_reg = VotingRegressor(
    estimators=[('Ex', Ex_reg), ('gbrt', gbrt_reg)], n_jobs = -1)
voting_reg.fit(x_train, housing_label)

VotingRegressor(estimators=[('Ex',
                             ExtraTreesRegressor(max_depth=30, n_estimators=800,
                                                 n_jobs=-1, random_state=42)),
                            ('gbrt',
                             GradientBoostingRegressor(max_depth=2,
                                                       n_estimators=4100,
                                                       random_state=42))],
                n_jobs=-1)

## Voting의 rmse값 출력

In [25]:
housing_predictions_voting = voting_reg.predict(x_train)
voting_mse = mean_squared_error(housing_label, housing_predictions_voting)
voting_rmse = np.sqrt(voting_mse)
voting_rmse

37402.6327772827

## voting의 10-fold cross-validation 실행 후, Scores, 평균오차, 표준오차 출력

In [26]:
voting_scores = cross_val_score(voting_reg, x_train, housing_label,
                                scoring="neg_mean_squared_error", cv=10)
voting_rmse_scores = np.sqrt(-voting_scores)
display_scores(voting_rmse_scores)

Scores: [115297.01669468 125901.18684871 106335.72176673 113327.21751575
  97359.74332268 119174.25184423 119179.4793509  107895.14712924
 109304.15762515 122139.84109429]
Mean: 113591.3763192347
Standard deviation: 8105.0587456356025


# Voting의 평균오차(Mean) : 113591.3763192347